<a href="https://colab.research.google.com/github/farookfazni/University_Id_Card_Detector/blob/main/ImageProcessing_Id_Card_Detector.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers

In [2]:
import numpy as np
import matplotlib.pyplot as plt

In [3]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.preprocessing import image
from tensorflow.keras.optimizers import RMSprop
import cv2
import os

In [4]:
# train = ImageDataGenerator(rescale= 1/255)
# validation = ImageDataGenerator(rescale= 1/255)

In [5]:
# train_dataset = train.flow_from_directory('drive/MyDrive/GoogleColab/dataset/train/', 
#                                           target_size=(1029, 644), 
#                                           batch_size = 3, 
#                                           class_mode = 'binary')
# validation_dataset = train.flow_from_directory('drive/MyDrive/GoogleColab/dataset/validation/', 
#                                           target_size=(1029, 644), 
#                                           batch_size = 3, 
#                                           class_mode = 'binary')

In [6]:
# train_dataset.class_indices

In [7]:
# train_dataset.classes

In [8]:
import pathlib
train_data_dir = pathlib.Path('drive/MyDrive/GoogleColab/dataset/train/')
train_data_dir

PosixPath('drive/MyDrive/GoogleColab/dataset/train')

In [9]:
image_count = len(list(train_data_dir.glob('*/*.jpg')))
print(image_count)

30


In [10]:
id_card_dict = {
    'id': list(train_data_dir.glob('id/*')),
    'not_id': list(train_data_dir.glob('not_id/*')),
}

In [11]:
id_card_labels_dict = {
    'id': 0,
    'not_id': 1,
}

In [12]:
X, y = [], []

for id_name, images in id_card_dict.items():
    for image in images:
        img = cv2.imread(str(image))
        resized_img = cv2.resize(img,(1029, 644))
        X.append(resized_img)
        y.append(id_card_labels_dict[id_name])

In [13]:
X = np.array(X)
y = np.array(y)

In [14]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=0)

In [15]:
X_train_scaled = X_train / 255
X_test_scaled = X_test / 255

In [16]:
data_augmentation = keras.Sequential(
  [
    keras.layers.experimental.preprocessing.RandomFlip("horizontal",input_shape=(644,1029,3)),
    keras.layers.experimental.preprocessing.RandomRotation(0.1),
    keras.layers.experimental.preprocessing.RandomZoom(0.1),
  ]
)

In [17]:
model = keras.Sequential([
                           data_augmentation,
                           keras.layers.Conv2D(8, 3, padding='same', activation='relu'),
                           keras.layers.MaxPool2D(),
                           #
                           keras.layers.Conv2D(16, 3, padding='same',activation='relu'),
                           keras.layers.MaxPool2D(),
                           #
                           keras.layers.Conv2D(32 ,3, padding='same', activation='relu'),
                           keras.layers.MaxPool2D(),
                           keras.layers.Dropout(0.2),
                           ##
                           keras.layers.Flatten(),
                           ##
                           keras.layers.Dense(10,activation='relu'),
                           ##
                           keras.layers.Dense(1,activation='sigmoid')
])

In [18]:
model.compile(optimizer='adam',
              loss='BinaryCrossentropy',
              metrics=['accuracy'])

In [ ]:
model.fit(X_train_scaled, y_train, epochs=15)

Epoch 1/15
2/2 [==============================] - 26s 10s/step - loss: 0.9546 - accuracy: 0.5818
Epoch 2/15
2/2 [==============================] - 25s 10s/step - loss: 0.8504 - accuracy: 0.6182
Epoch 3/15
2/2 [==============================] - 25s 10s/step - loss: 0.6434 - accuracy: 0.8182
Epoch 4/15
2/2 [==============================] - 25s 10s/step - loss: 0.6023 - accuracy: 0.6727
Epoch 5/15
2/2 [==============================] - 25s 10s/step - loss: 0.5604 - accuracy: 0.6364
Epoch 6/15
2/2 [==============================] - 25s 10s/step - loss: 0.5358 - accuracy: 0.6364
Epoch 7/15
2/2 [==============================] - 28s 13s/step - loss: 0.5072 - accuracy: 0.6545
Epoch 8/15
2/2 [==============================] - 25s 10s/step - loss: 0.4784 - accuracy: 0.7455
Epoch 9/15
2/2 [==============================] - 25s 10s/step - loss: 0.4516 - accuracy: 0.8000
Epoch 10/15
2/2 [==============================] - 25s 10s/step - loss: 0.4135 - accuracy: 0.8182
Epoch 11/15
2/2 [============

In [ ]:
model.evaluate(X_test_scaled,y_test)

1/1 [==============================] - 3s 3s/step - loss: 0.3067 - accuracy: 0.8421


[0.30667489767074585, 0.8421052694320679]

In [20]:
pip install -q tensorflow-model-optimization

     |████████████████████████████████| 211 kB 4.1 MB/s 


In [21]:
pip install --user --upgrade tensorflow-model-optimization

In [22]:
print(tf. __version__)

2.6.0


In [23]:
import tensorflow_model_optimization as tfmot

In [24]:
quantize_model = tfmot.quantization.keras.quantize_annotate_model

In [25]:
quantize_annotate_layer = tfmot.quantization.keras.quantize_annotate_layer

In [26]:
quantize_apply = tfmot.quantization.keras.quantize_apply

In [27]:
quantize_config = tfmot.quantization.keras.QuantizeConfig

In [32]:
model2 = keras.Sequential([
                           quantize_annotate_layer(keras.layers.experimental.preprocessing.RandomFlip("horizontal",input_shape=(644,1029,3))),
                           quantize_annotate_layer(keras.layers.experimental.preprocessing.RandomRotation(0.1)),
                           quantize_annotate_layer(keras.layers.experimental.preprocessing.RandomZoom(0.1)),
                           quantize_annotate_layer(keras.layers.Conv2D(8, 3, padding='same', activation='relu')),
                           quantize_annotate_layer(keras.layers.MaxPool2D()),
                           #
                           quantize_annotate_layer(keras.layers.Conv2D(16, 3, padding='same',activation='relu')),
                           quantize_annotate_layer(keras.layers.MaxPool2D()),
                           #
                           quantize_annotate_layer(keras.layers.Conv2D(32 ,3, padding='same', activation='relu')),
                           quantize_annotate_layer(keras.layers.MaxPool2D()),
                           keras.layers.Dropout(0.2),
                           ##
                           keras.layers.Flatten(),
                           ##
                           quantize_annotate_layer(keras.layers.Dense(10,activation='relu')),
                           ##
                           keras.layers.Dense(1,activation='sigmoid')
])

In [33]:
q_aware_model = quantize_model(model2)

In [34]:
q_aware_model.compile(optimizer='adam',
              loss='BinaryCrossentropy',
              metrics=['accuracy'])
q_aware_model.summary()

Model: "sequential_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
quantize_annotate_13 (Quanti (None, 644, 1029, 3)      0         
_________________________________________________________________
quantize_annotate_14 (Quanti (None, 644, 1029, 3)      0         
_________________________________________________________________
quantize_annotate_15 (Quanti (None, 644, 1029, 3)      0         
_________________________________________________________________
quantize_annotate_16 (Quanti (None, 644, 1029, 8)      224       
_________________________________________________________________
quantize_annotate_17 (Quanti (None, 322, 514, 8)       0         
_________________________________________________________________
quantize_annotate_18 (Quanti (None, 322, 514, 16)      1168      
_________________________________________________________________
quantize_annotate_19 (Quanti (None, 161, 257, 16)     

In [39]:
q_aware_model.fit(X_train_scaled, y_train, epochs=10, steps_per_epoch=10)

Epoch 1/10
10/10 [==============================] - 25s 2s/step - loss: 0.4523 - accuracy: 0.7273
Epoch 2/10
10/10 [==============================] - 25s 2s/step - loss: 0.3373 - accuracy: 0.8182
Epoch 3/10
10/10 [==============================] - 24s 2s/step - loss: 0.3013 - accuracy: 0.8727
Epoch 4/10
10/10 [==============================] - 25s 2s/step - loss: 0.1982 - accuracy: 0.9455
Epoch 5/10
10/10 [==============================] - 25s 2s/step - loss: 0.1708 - accuracy: 0.9273
Epoch 6/10
10/10 [==============================] - 25s 2s/step - loss: 0.1386 - accuracy: 0.9455
Epoch 7/10
10/10 [==============================] - 25s 2s/step - loss: 0.1059 - accuracy: 0.9636
Epoch 8/10
10/10 [==============================] - 24s 2s/step - loss: 0.2313 - accuracy: 0.9273
Epoch 9/10
10/10 [==============================] - 24s 2s/step - loss: 0.1398 - accuracy: 0.9636
Epoch 10/10
10/10 [==============================] - 24s 2s/step - loss: 0.1368 - accuracy: 0.9455


In [40]:
q_aware_model.evaluate(X_test_scaled,y_test)

1/1 [==============================] - 2s 2s/step - loss: 0.1077 - accuracy: 0.9474


[0.10771586745977402, 0.9473684430122375]

In [41]:
converter = tf.lite.TFLiteConverter.from_keras_model(q_aware_model)
converter.optimizations = [tf.lite.Optimize.DEFAULT]
tflite_q_qwre_model = converter.convert()

INFO:tensorflow:Assets written to: /tmp/tmpjxq_1gpv/assets


INFO:tensorflow:Assets written to: /tmp/tmpjxq_1gpv/assets


In [43]:
with open('tflite_q_qwre_model.tflite','wb') as f:
  f.write(tflite_q_qwre_model)

In [45]:
interpreter = tf.lite.Interpreter(model_path="tflite_q_qwre_model.tflite")
interpreter.allocate_tensors()

# Get input and output tensors.
input_details = interpreter.get_input_details()
output_details = interpreter.get_output_details()

In [ ]:
input_details[0]